In [1]:
import numpy as np
import random
import torch
from src.dataloaders import Dataset100k, Dataset1m
from src.models import GMFBCEModel, MLPBCEModel, NeuralMatrixFactorizationBPRModel
from src.trainer import Trainer
from src.metrics import hitratio, ndcg

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = 'cpu'
print(f"{device=}")

device='cuda'


In [2]:
class config:
    data_dir = "ml-100k"
    # data_dir = 'ml-1m'
    epochs = 10
    batch_size = 2048
    gmf_embed_size = 16
    mlp_embed_size = 32
    layers = [32, 16, 8]
    lr = 0.001
    b1 = 0.7
    b2 = 0.5
    weight_decay = 0.0001

In [3]:
dataset = Dataset100k(config.data_dir)
# dataset = Dataset1m(config.data_dir)
dataset.gen_adjacency()
dataset.make_train_test()
print(f"{dataset.train_size=}, {dataset.test_size=}")

metrics = {
    "HR@1": (hitratio, {"top_n": 1}),
    "HR@5": (hitratio, {"top_n": 5}),
    "HR@10": (hitratio, {"top_n": 10}),
    "NDCG@1": (ndcg, {"top_n": 1}),
    "NDCG@5": (ndcg, {"top_n": 5}),
    "NDCG@10": (ndcg, {"top_n": 10}),
}

dataset.train_size=198114, dataset.test_size=943


In [4]:
model = NeuralMatrixFactorizationBPRModel(
    dataset.user_count,
    dataset.item_count,
    gmf_embed_size=config.gmf_embed_size,
    mlp_embed_size=config.mlp_embed_size,
    layers=config.layers,
    alpha=0.5,
)

gmf_model = GMFBCEModel(
    dataset.user_count, dataset.item_count, embed_size=config.gmf_embed_size
)
gmf_model.load_state_dict(torch.load("saved_models/gmfbce.pt"))
mlp_model = MLPBCEModel(
    dataset.user_count,
    dataset.item_count,
    embed_size=config.mlp_embed_size,
    layers=config.layers,
)
mlp_model.load_state_dict(torch.load("saved_models/mlpbce.pt"))

model.load_pretrained_weights(gmf_model, mlp_model)
del gmf_model
del mlp_model

# optimizer = torch.optim.SGD(model.parameters(), lr=config.lr, momentum=config.momentum, nesterov=True, weight_decay=config.weight_decay)
# optimizer = torch.optim.SGD(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=config.lr,
    betas=(config.b1, config.b2),
    weight_decay=config.weight_decay,
)
# optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)

trainer = Trainer(
    dataset,
    model,
    optimizer,
    metrics,
    epochs=config.epochs,
    batch_size=config.batch_size,
    device=device,
)

In [5]:
trainer.train(evaluate=True, verbose=True, progressbar=True)
# trainer.test(verbose=False, pbar=False)

Epoch 0: Avg Loss/Batch 344.946300          


HR@1: 0.18451749734888653
HR@5: 0.3669141039236479
HR@10: 0.4687168610816543
NDCG@1: 0.18451749734888653
NDCG@5: 0.2795104538869772
NDCG@10: 0.3120651023575053


Epoch 1: Avg Loss/Batch 148.867295          


HR@1: 0.33510074231177095
HR@5: 0.36585365853658536
HR@10: 0.41675503711558853
NDCG@1: 0.33510074231177095
NDCG@5: 0.35002985049783253
NDCG@10: 0.3667948034643804


Epoch 2: Avg Loss/Batch 148.864229          


...

HR@1: 0.026511134676564158
HR@5: 0.026511134676564158
HR@10: 0.026511134676564158
NDCG@1: 0.026511134676564158
NDCG@5: 0.026511134676564158
NDCG@10: 0.026511134676564158


Epoch 8: Avg Loss/Batch 148.864192          


HR@1: 0.32343584305408274
HR@5: 0.32343584305408274
HR@10: 0.32343584305408274
NDCG@1: 0.32343584305408274
NDCG@5: 0.32343584305408274
NDCG@10: 0.32343584305408274


Epoch 9: Avg Loss/Batch 148.864191          


HR@1: 0.264050901378579
HR@5: 0.2651113467656416
HR@10: 0.2926829268292683
NDCG@1: 0.264050901378579
NDCG@5: 0.26446113765348306
NDCG@10: 0.27311728095506826


In [10]:
best_epoch = np.argmax([r["NDCG@10"] for r in trainer.test_log])
print(f"{best_epoch}: {trainer.test_log[best_epoch]}")

6: {'HR@1': 0.38494167550371156, 'HR@5': 0.38494167550371156, 'HR@10': 0.38494167550371156, 'NDCG@1': 0.38494167550371156, 'NDCG@5': 0.38494167550371156, 'NDCG@10': 0.38494167550371156}


In [11]:
torch.save(trainer.model.state_dict(), "saved_models/nmfbpr.pt")
# trainer.model.load_state_dict(torch.load("saved_models/nmfbpr.pt"))